In [1]:

%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
# import os
# print(os.getpid())
# %cd ../

from numpy import linalg 

In [2]:
for i in range(30):
    plt.close()

In [3]:
X_gate = np.array([
    [0.0, 1.0],
    [1.0, 0.0]
])

H_gate = np.array([
    [1.0, 1.0],
    [1.0, -1.0]
]) / np.sqrt(2)

Ry_gate = lambda a: np.array([
    [np.cos(a/2.), -np.sin(a/2.)],
    [np.sin(a/2.), np.cos(a/2.)]
])


In [4]:
from numba import jit

@jit(nopython=True)
def compare_matrices_dense(B, A, prec = 1e-6, flag_return_err = False):
    N = A.shape[0]
    if N != B.shape[0]:
        print("The matrices have different sizes.")
        return

    max_abs_err = 0
    for ir in range(N):
        for ic in range(N):
            ar, ai = np.real(B[ir, ic]), np.imag(B[ir, ic])
            br, bi = np.real(A[ir, ic]), np.imag(A[ir, ic])

            # print("ir, ic, A, B: ", ir, ic, A[ir, ic], B[ir, ic])

            if flag_return_err:
                err_loc = np.abs(B[ir, ic] - A[ir, ic])
                if err_loc > max_abs_err:
                    max_abs_err = err_loc
            else:
                flag_not_the_same = False
                if np.abs(ar - br) > prec:
                    flag_not_the_same = True
                if np.abs(ai - bi) > prec:
                    flag_not_the_same = True

                if flag_not_the_same:
                    print(\
                        "WARNING: the matrices are NOT the same within" + \
                            " the following precision: ", prec
                    )
                    return
    if flag_return_err:
        # print("max. abs. err: {:0.3e}".format(max_abs_err))
        if max_abs_err <= 1e-21:
            max_abs_err = 1e-21
        print("log10|max.err|: ", np.log10(max_abs_err))
    else:
        # print("The matrices are the same within the following precision: {:0.3e}".format(prec))
        print("The matrices are the same within the following precision: ", prec)
    return

In [5]:
# -----------------------------------------------
# --- Construct a circuit ---
# -----------------------------------------------
from qiskit import QuantumCircuit
from qiskit.circuit.library import HGate, UnitaryGate
import qiskit.quantum_info as qi
from qiskit.quantum_info.operators import Operator
import numpy as np
import random

# --------------------------------------------------------------------
def rec_matrix(evecs, evals_diag):
    U1 = np.dot(evecs, evals_diag)
    U1 = np.dot(U1, linalg.inv(evecs))
    return U1


# --------------------------------------------------------------------
def build_U_sqrt(U_orig):
    evals, evecs = linalg.eig(U_orig)
    evals_diag = np.array(np.diag(evals), dtype=complex)
    sqrt_evals = np.sqrt(evals_diag)
    U_sqrt = rec_matrix(evecs, sqrt_evals)
    return U_sqrt


# --------------------------------------------------------------------
def decomposition_gate(qc, targ_V, nc_curr, Us_sq, Xs_sq, id_U, flag_X = False):
    targ_X = nc_curr - 1

    if flag_X:
        str_gate = "X"
    else:
        str_gate = "U"
    U1_sq = Us_sq[id_U]
    id_U += 1
    Ucc = np.conjugate(np.transpose(U1_sq))

    mcU  = UnitaryGate(U1_sq, str_gate).control(1)
    mcUi = UnitaryGate(Ucc, "i"+str_gate).control(1)

    if nc_curr == 2:
        qc.cx(0, targ_X) 
        qc.append(mcUi, [targ_X, targ_V])
        qc.cx(0, targ_X)
        qc.append(mcU, [targ_X, targ_V]) 
        qc.append(mcU,  [0, targ_V])
    else:
        nc_next = nc_curr-1
        decomposition_gate(qc, targ_X, nc_next, Xs_sq, Xs_sq,    0, True)
        qc.append(mcUi, [targ_X, targ_V])
        decomposition_gate(qc, targ_X, nc_next, Xs_sq, Xs_sq,    0, True)
        qc.append(mcU,  [targ_X, targ_V]) 

        # decomposition_gate(qc, targ_V, nc_next, Us_sq, Xs_sq, id_U, flag_X) 

        if not flag_X:
            if id_U < (n_iter_stop_ - 1):
                decomposition_gate(qc, targ_V, nc_next, Us_sq, Xs_sq, id_U, flag_X) 
        else:
            decomposition_gate(qc, targ_V, nc_next, Us_sq, Xs_sq, id_U, flag_X) 
    return

# --------------------------------------------------------------------

nc = 5             # the number of control qubits 
sel_case = 2       # choose the gate to decompose
n_iter_stop_ = 30  # roots higher or equal than n_iter_stop are neglected

if sel_case == 0:
    U = np.array(X_gate)
    str_gate = "X"

if sel_case == 1:
    U = np.array(H_gate)
    str_gate = "H"

if sel_case == 2:
    aa = random.uniform(-np.pi/2., np.pi/2.)
    U = np.array(Ry_gate(aa))
    str_gate = "Ry({:0.3e})".format(aa)

# --- Decompose the gate ---
Us_sq = np.zeros((nc-1, 2, 2), dtype=complex)
U_curr = np.array(U)
for i_iter in range(nc-1):
    U_sqrt = build_U_sqrt(U_curr)
    Us_sq[i_iter, :, :] = U_sqrt
    U_curr = U_sqrt
del U_curr, U_sqrt

# --- Decompose a multicontrolled X-gate            ---
# --- (necessary for the decomposition of any gate) ---
Xs_sq = None
if nc > 2:
    Xs_sq = np.zeros((nc-2, 2, 2), dtype=complex)
    U_curr = np.array(X_gate)
    for i_iter in range(nc-2):
        U_sqrt = build_U_sqrt(U_curr)
        Xs_sq[i_iter, :, :] = U_sqrt
        U_curr = U_sqrt
    del U_curr, U_sqrt

# ---
nq = nc + 1
rc = list(range(nc))
rq = list(range(nc+1))

# ---------------------------------------------------------------
# --- Reference circuit ---
print()
print("---------------------------------------------------")
print("--- Reference circuit. ---")
qc_ref = QuantumCircuit(nq)
mcU = UnitaryGate(U, label = str_gate).control(nc)
qc_ref.append(mcU, rq)
print(qc_ref.draw())
print("REF. circuit's depth: ", qc_ref.depth())

# ---------------------------------------------------------------
# --- Decomposed circuit ---
print()
print("---------------------------------------------------")
print("--- Circuit for the decomposed gate. ---")
qc_dec = QuantumCircuit(nq)
decomposition_gate(qc_dec, nq-1, nc, Us_sq, Xs_sq, 0)
# print(qc_dec.draw(fold=300))
print("DEC. circuit's depth: ", qc_dec.depth())

# --- Compare two circuits ---
print()
print("Compare circuits")
U_ref = qi.Operator(qc_ref).data
U_dec = qi.Operator(qc_dec).data
compare_matrices_dense(U_ref, U_dec, flag_return_err=True)


---------------------------------------------------
--- Reference circuit. ---
                      
q_0: ────────■────────
             │        
q_1: ────────■────────
             │        
q_2: ────────■────────
             │        
q_3: ────────■────────
             │        
q_4: ────────■────────
     ┌───────┴───────┐
q_5: ┤ Ry(9.774e-01) ├
     └───────────────┘
REF. circuit's depth:  1

---------------------------------------------------
--- Circuit for the decomposed gate. ---
DEC. circuit's depth:  135

Compare circuits
log10|max.err|:  -12.9003559492732


In [ ]:
# -------------------------------------------------
# --- Scan: Depth versus nc ---
# Here, single-target and single-controlled single-target unitary-gates are NOT
#   decomposed into universal gates.
# -------------------------------------------------
nc_array = [2,  3,  4,   5,   6,  7]
D_array  = [5, 15, 45, 135, 405,  1215]
